# Data

In [ ]:
import tensorflow as tf
from google.colab.patches import cv2_imshow
from PIL import Image
import os
import numpy as np

In [ ]:
!pip install opencv-python==4.5.4.60

In [ ]:
import cv2 as cv

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle

  Using cached kaggle-1.5.12-py3-none-any.whl
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [ ]:
from google.colab import files

Здесь (в следующей ячейке) нужно загрузить kaggle api ключ как .json файл, его можно получить в найстройках аккаунта на сайте kaggle.com

In [ ]:
files.upload()

In [ ]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets list -s {,,'stitching'}

ref                                                    title                                               size  lastUpdated          downloadCount  voteCount  usabilityRating  
-----------------------------------------------------  --------------------------------------------------  ----  -------------------  -------------  ---------  ---------------  
phsophea101/image-for-stitching                        Image for stitching in computer vision (cv2)python   2GB  2021-03-07 15:58:51             32          5  0.5625           
phsophea101/image-stitching-from-drone-capture-opencv  Image stitching from drone capture OpenCV           64MB  2021-03-21 08:38:29             44          2  0.4375           


In [ ]:
!kaggle datasets download -d {,,'phsophea101/image-stitching-from-drone-capture-opencv'} -p /content/kaggle/

image-stitching-from-drone-capture-opencv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip '/content/kaggle/image-stitching-from-drone-capture-opencv.zip' -d '/content/kaggle/'

Archive:  /content/kaggle/image-stitching-from-drone-capture-opencv.zip
replace /content/kaggle/drone/image_0001.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


## reviewing images

In [ ]:
def print_image(n):
  img = cv.imread('/content/kaggle/drone/image_'+n+'.jpg')
  cv2_imshow(img)

In [ ]:
print_image('0001')

In [ ]:
print_image('0021')

In [ ]:
images = [f for f in os.listdir('/content/kaggle/drone')]
images.sort()
arr_images = []
for i in images:
    arr_images.append(cv.imread('/content/kaggle/drone/'+i))
arr_images = np.array(arr_images)

In [ ]:
arr_images.shape

(44, 1530, 2720, 3)

# first step

In [ ]:
?cv.drawKeypoints

In [ ]:
import cv2 as cv
img = arr_images[0]
#img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
sift = cv.SIFT_create()
keypoints, descriptors = sift.detectAndCompute(img,None)
img = cv.drawKeypoints(img, keypoints, None)
print(len(keypoints))
cv2_imshow(img)

In [ ]:
img1 = arr_images[0]
img2 = arr_images[2]
sift = cv.SIFT_create(200)

keypoints_1, descriptors_1 = sift.detectAndCompute(img1,None)
keypoints_2, descriptors_2 = sift.detectAndCompute(img2,None)
bf = cv.BFMatcher(cv.NORM_L1, crossCheck=True)

matches = bf.match(descriptors_1,descriptors_2)
matches = sorted(matches, key = lambda x:x.distance)

img3 = cv.drawMatches(img1, keypoints_1, img2, keypoints_2, matches[:50], img2, flags=2)
cv2_imshow(img3)

In [ ]:
stitcher = cv.Stitcher.create()
(status, result) = stitcher.stitch(arr_images[:20])
if (status == cv.STITCHER_OK):
  print('Success!')
  cv2_imshow(result)
else:
  print('Dumpt')

Выводы:
Алгоритм работает хорошо, на статичных снимках (когда перемещается камера относительно объектов). Если на снимках есть динамические объект, например: передвигающийся автомобиль вместе с передвижением камеры (и изминением пейзажа) и тд - алгоритму детекции точек и дисприпторов сложно вычленит фон, потому он привязывается к статичному объекту относительно камеры (т.е. динамическому относительно фона). Возможные улучшения и доработки: предобрабатываться такие "сложные" картинки, средством удаления статичных объектов относительно камеры, а уже после использовать алгоритм( искать ключевые точки, дескрипторы, связи между ними, матрицу гомографии, и склейку изображений)